characters를 인덱스로 표현. continuous하지 않고 categorical 한 data를 표현할 때는 주로 one-hot-encoding 방식을 사용해라.

In [7]:
import torch
import numpy as np

char_set=['h','i','e','l','o']
x_data=[[0,1,0,2,3,3]]
x_one_hot=[[[1,0,0,0,0],
            [0,1,0,0,0],
            [1,0,0,0,0],
            [0,0,1,0,0],
            [0,0,0,1,0],
            [0,0,0,1,0]
            ]]              # 마지막의 o는 없다. 왜와이? 다음 문자를 예측하는 모델이므로 입력값에는 o값 이전의 값들을 넣는다.

y_data=[[1,0,2,3,3,4]]      # 얘는 o가 포함되었다. 다만 첫번째 h를 넣지 않았다.


Loss for categorical output(usually interpreted as probability)

In [8]:
criterion=torch.nn.CrossEntropyLoss()
loss=criterion(outputs.view(-1,input_size), Y.view(-1))             #outputs는 모델의 output이고, Y는 정답레이블이다. 이 순서를 바꾸지 마라.
                                                                    #참고) outputs는 원래 shape이 N,T,H이고 view를 하면 N*T,H가 되고
                                                                    #Y는 원래 (N,T)인데 이걸 view를 하면 Y가 일차원 벡터로 바뀌어서 (N*T,)가 된다.

NameError: name 'outputs' is not defined

In [9]:
# Code run through (hi hello)

char_set=['h','i','e','l','o']

#hyper parameters
input_size=len(char_set)
hidden_size=len(char_set)           # 이번에는 input_size와 같게 하지만 같을 필요는 없다.
learning_rate=0.1

#data setting
x_data=[[0,1,0,2,3,3]]
x_one_hot=[[[1,0,0,0,0],
            [0,1,0,0,0],
            [1,0,0,0,0],
            [0,0,1,0,0],
            [0,0,0,1,0],
            [0,0,0,1,0]
            ]]  
y_data=[[1,0,2,3,3,4]]


#transform as torch tensor variable
X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)

In [10]:
# 이번에는 charseq를 해보자.
# 데이터 준비과정만 조금 차이가 있고 나머지는 거의 동일하다.

sample=' if you want you'

#make dicitonary
char_set=list(set(sample))                      #unique한 char만 모음.
char_dic={c: i for i,c in enumerate(char_set)}

#hyper parameters
dic_size=len(char_dic)
hidden_size=len(char_dic)
learning_rate=0.1

#data setting
sample_idx=[char_dic[c] for c in sample]
x_data=[sample_idx[:-1]]                         # 마지막 char는 제외함.
x_one_hot=[np.eye(dic_size)[x] for x in x_data]  # np.eye를 하면 dic_size의 identity matrix를 만들 수 있다.
y_data=[sample_idx[1:]]                          # 맨 처음만 뺀다.

#transform as torch tensor variable
X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)

C:\Users\dhlee\AppData\Local\Temp\ipykernel_7364\2794592490.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X=torch.FloatTensor(x_one_hot)


In [ ]:
# Code run through

# declare RNN
rnn=torch.nn.RNN(dic_size, hidden_size, batch_first=True)             # batch_first guarantees the order of output=(B,S,F)

# loss & optimizer setting
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(rnn.parameters(),learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()                       #매 루프마다 새로운 grad를 구함. 해주지 않으면 기존의 grad가 축적되어 잘못된 grad를 구해서 training이 잘 되지않음.
    outputs, _status=rnn(X)                     #_status는 주어진 input을 다 처리하고 나온 hidden_state. 따로 쓰지는 않는다. 
    loss=criterion(outputs.view(-1,dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    result=outputs.data.numpy().argmax(axis=2)
    result_str=''.join([char_set[c] for c in np.squeeze(result)])       #np.squeeze()를 사용하면 크기가 1인 차원인 1을 제거한한 형태로 반환
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediciton str: ", result_str)

0 loss:  2.2591123580932617 prediction:  [[1 9 9 7 4 6 4 7 9 5 9 9 7 4 6]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediciton str:  owwynfnywawwynf
1 loss:  1.9596996307373047 prediction:  [[7 9 2 7 1 3 1 7 1 7 1 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediciton str:  yw youoyoyo you
2 loss:  1.7488327026367188 prediction:  [[7 5 2 7 1 3 1 7 5 7 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediciton str:  ya youoyayt you
3 loss:  1.5983426570892334 prediction:  [[7 5 2 7 1 3 1 7 5 4 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediciton str:  ya youoyant you
4 loss:  1.4599717855453491 prediction:  [[7 5 2 7 1 3 1 7 5 4 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediciton str:  ya youoyant you
5 loss:  1.3238192796707153 prediction:  [[7 6 2 7 1 3 1 9 5 4 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediciton str:  yf youowant you
6 loss:  1

통합

In [12]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

sample = " if you want you"

# make dictionary
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

# start training
for i in range(50):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  2.2467610836029053 prediction:  [[8 5 7 8 0 0 7 8 6 7 7 7 5 0 5]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediction str:  taytiiytfyyyaia
1 loss:  1.9019049406051636 prediction:  [[7 1 2 8 1 3 7 8 1 2 8 7 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediction str:  yo touyto tyyou
2 loss:  1.6627609729766846 prediction:  [[7 6 2 7 1 3 7 8 6 2 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediction str:  yf youytf t you
3 loss:  1.5160969495773315 prediction:  [[7 6 2 7 1 3 7 9 6 2 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediction str:  yf youywf t you
4 loss:  1.4335267543792725 prediction:  [[7 6 2 7 1 3 7 9 5 7 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediction str:  yf youywayt you
5 loss:  1.3486058712005615 prediction:  [[7 6 2 7 1 3 2 9 5 2 8 2 7 1 3]] true Y:  [[0, 6, 2, 7, 1, 3, 2, 9, 5, 4, 8, 2, 7, 1, 3]] prediction str:  yf you wa t you
6 loss:  1